In [1]:

import torch
import torch.nn as nn
from torchvision import models
from PIL import Image
import shap
import numpy as np
import os

KeyboardInterrupt: 

In [ ]:
device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
model_path = r"C:\Users\Ekaansh\OneDrive\Desktop\AB\research\SHAP\best_pneumonia_densenet121.pt"
class_names = ['NORMAL', 'PNEUMONIA']

In [ ]:
model = models.densenet121(weights=None)
num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, len(class_names))
)
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()

C:\Users\Ekaansh\AppData\Local\Temp\ipykernel_11740\832990262.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
import shap
import numpy as np
from PIL import Image

In [ ]:
class_names = ['NORMAL', 'PNEUMONIA']

In [ ]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [ ]:
def preprocess_input(images):

    images = images / 255.0
    images = (images - mean) / std
    # convert to PyTorch CHW
    images = images.transpose(0, 3, 1, 2)
    return torch.tensor(images).float()

In [ ]:
def f(x):
    
    x_tensor = preprocess_input(x)
    with torch.no_grad():
        out = model(x_tensor)
        probs = torch.softmax(out, dim=1)
    return probs.cpu().numpy() 

In [ ]:
all_images = [
    r"C:\Users\Ekaansh\OneDrive\Desktop\AB\research\SHAP\finding\shap_pred_1"
]

image_paths = all_images[:50]
X = np.stack([np.array(Image.open(p).convert("RGB").resize((224, 224))) for p in image_paths])


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Ekaansh\\OneDrive\\Desktop\\AB\\research\\SHAP\\finding\\shap_pred_1'

In [ ]:
masker = shap.maskers.Image("inpaint_telea", X[0].shape)




In [ ]:
explainer = shap.Explainer(f, masker, output_names=class_names)

In [ ]:
shap_values = explainer(
    X,                 
    max_evals=10000,          
    batch_size=50,          
    outputs=shap.Explanation.argsort.flip[:2]
)

In [ ]:
shap.image_plot(shap_values)